<a href="https://colab.research.google.com/github/NoelGit2003/Deep_Fake_Video_Classification/blob/main/Deep_Fake_Videos_Classification_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install mtcnn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 9.8 MB/s eta 0:00:00


In [9]:
import pandas as pd
import pathlib
import cv2
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import ThreadPoolExecutor
from mtcnn import MTCNN
import os
import shutil
import warnings
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#warnings.filterwarnings("ignore")

In [ ]:
#os.environ["CUDA_VISIBLE_DEVICES"] = '-1'

In [ ]:
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '4'

In [ ]:
!pwd

/content


In [ ]:
!wget https://zenodo.org/record/4068245/files/DeepfakeTIMIT.tar.gz?download=1

--2024-02-10 09:30:03--  https://zenodo.org/record/4068245/files/DeepfakeTIMIT.tar.gz?download=1
Resolving zenodo.org (zenodo.org)... 188.184.98.238, 188.184.103.159, 188.185.79.172, ...
Connecting to zenodo.org (zenodo.org)|188.184.98.238|:443... connected.
HTTP request sent, awaiting response... 301 MOVED PERMANENTLY
Location: /records/4068245/files/DeepfakeTIMIT.tar.gz [following]
--2024-02-10 09:30:04--  https://zenodo.org/records/4068245/files/DeepfakeTIMIT.tar.gz
Reusing existing connection to zenodo.org:443.
HTTP request sent, awaiting response... 200 OK
Length: 226611200 (216M) [application/octet-stream]
Saving to: ‘DeepfakeTIMIT.tar.gz?download=1’

DeepfakeTIMIT.tar.g 100%[===================>] 216.11M  17.4MB/s    in 14s     

2024-02-10 09:30:19 (15.2 MB/s) - ‘DeepfakeTIMIT.tar.gz?download=1’ saved [226611200/226611200]



In [ ]:
!tar -xf ./DeepfakeTIMIT.tar.gz?download=1

In [ ]:
!wget https://lp-prod-resources.s3.amazonaws.com/other/detectingdeepfakes/VidTIMIT.zip

--2024-02-10 09:30:20--  https://lp-prod-resources.s3.amazonaws.com/other/detectingdeepfakes/VidTIMIT.zip
Resolving lp-prod-resources.s3.amazonaws.com (lp-prod-resources.s3.amazonaws.com)... 52.92.162.225, 52.92.206.153, 52.92.197.113, ...
Connecting to lp-prod-resources.s3.amazonaws.com (lp-prod-resources.s3.amazonaws.com)|52.92.162.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1358810924 (1.3G) [application/zip]
Saving to: ‘VidTIMIT.zip’

VidTIMIT.zip        100%[===================>]   1.26G  21.0MB/s    in 31s     

2024-02-10 09:30:51 (41.7 MB/s) - ‘VidTIMIT.zip’ saved [1358810924/1358810924]



In [ ]:
!unzip -qq ./VidTIMIT.zip

In [ ]:
!rm -rf ./__MACOSX

In [ ]:
!mkdir -p ./Training\ Videos/preproc_real_vids
!mkdir -p ./Training\ Videos/preproc_fake_vids

In [ ]:
!mkdir -p ./Testing\ Videos/preproc_real_vids
!mkdir -p ./Testing\ Videos/preproc_fake_vids

In [ ]:
!mkdir ./Remaining\ Real\ Videos

In [ ]:
real_fake_vids_paths = ["./DeepfakeTIMIT/higher_quality","./VidTIMIT"]

In [ ]:
for base_vid_path in real_fake_vids_paths:

    vid_counter = 0
    for single_vid_path in pathlib.Path(base_vid_path).glob("*/*.avi"):

        vid_counter += 1

    print("Total Number of Videos are {}".format(vid_counter))
    vid_counter = 0

Total Number of Videos are 320
Total Number of Videos are 320


In [ ]:
vid_counter = 0

for single_vid_path in pathlib.Path(real_fake_vids_paths[1]).glob("*/*.avi"):

    if str(single_vid_path.parts[-2]) in os.listdir(real_fake_vids_paths[0]):
        vid_counter += 1

    else:
        if not os.path.isdir("./Remaining Real Videos/"+single_vid_path.parts[-2]):
            os.mkdir("./Remaining Real Videos/"+single_vid_path.parts[-2])

        shutil.move(src=single_vid_path,
                    dst="./Remaining Real Videos/"+"/".join([single_vid_path.parts[-2],
                                                           single_vid_path.parts[-1]]))

print("Total Number of Videos are {}".format(vid_counter))

Total Number of Videos are 320


In [ ]:
for base_vid_path in real_fake_vids_paths:

    vid_counter = 0
    for single_vid_path in pathlib.Path(base_vid_path).glob("*/*.avi"):

        vid_counter += 1

    print("Total Number of Videos are {}".format(vid_counter))

Total Number of Videos are 320
Total Number of Videos are 320


In [ ]:
def detect_face(frame):

    face_detector = MTCNN()
    face_attr = face_detector.detect_faces(frame)

    if len(face_attr) > 0:
        if len(face_attr[0]['box']) > 0:
            return face_attr[0]['box'], face_attr[0]['keypoints']['left_eye'], face_attr[0]['keypoints']['right_eye']
    else:
        return [None]*3

In [ ]:
def determine_rotation_angle(left_eye_kp, right_eye_kp):

    delta_y = right_eye_kp[1] - left_eye_kp[1]
    delta_x = right_eye_kp[0] - left_eye_kp[0]

    return np.arctan(delta_y/delta_x)

In [ ]:
def determine_scaling_factor(rescaled_img_size,left_eye_kp,right_eye_kp):

    left_eye_kp = np.array(left_eye_kp)
    right_eye_kp = np.array(right_eye_kp)

    whole_frame_eye_dist = np.linalg.norm(x=(right_eye_kp - left_eye_kp),ord=2)

    cropped_left_eye_kp = np.array([0.35*rescaled_img_size[0], 0.35*rescaled_img_size[1]])
    cropped_right_eye_kp = np.array([0.35*rescaled_img_size[0], 0.65*rescaled_img_size[1]])

    cropped_frame_eye_dist = np.linalg.norm(x=(cropped_right_eye_kp - cropped_left_eye_kp),
                                            ord=2)

    scaling_factor = cropped_frame_eye_dist/whole_frame_eye_dist

    return scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp

In [ ]:
def determine_rotation_center(left_eye_kp, right_eye_kp):

    center_x = (left_eye_kp[0] + right_eye_kp[0])/2
    center_y = (left_eye_kp[1] + right_eye_kp[1])/2

    return (center_x, center_y)

In [ ]:
"""
single_vid_path = pathlib.Path("./DeepfakeTIMIT/higher_quality/mrgg0/si569-video-msjs1.avi")

print("\n\nGoing to preprocess Video at {}".format(single_vid_path))

video = cv2.VideoCapture(str(single_vid_path))
cropped_aligned_faces_list = list()

while True:

    is_frame, frame = video.read()

    if not is_frame:
        break

    print("Going to preprocess Frame of {} having size {}".format(single_vid_path.parts[-1],
                                                                  frame.shape))

    face_bbox, left_eye_kp, right_eye_kp = detect_face(frame)

    if face_bbox == None:
        continue

    rotation_angle = determine_rotation_angle(left_eye_kp, right_eye_kp);
    scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp = determine_scaling_factor((224,224),
                                                                                            left_eye_kp, right_eye_kp)
    rotation_center = determine_rotation_center(left_eye_kp, right_eye_kp)

    #print("Computed everything to get the Rotation Matrix for the Frame")

    rotation_matrix = cv2.getRotationMatrix2D(center=rotation_center,
                                                         angle=rotation_angle,scale=scaling_factor)

    #print("Computed the Rotation Matrix for the Frame")

    cropped_center_x = cropped_left_eye_kp[0]
    cropped_center_y = cropped_left_eye_kp[1] + (np.linalg.norm(x=cropped_right_eye_kp-cropped_right_eye_kp,ord=2)/2)
    cropped_center = (cropped_center_x, cropped_center_y)

    rotation_matrix[0,2] = rotation_matrix[0,2] - (rotation_center[0] - cropped_center[0])
    rotation_matrix[1,2] = rotation_matrix[1,2] - (rotation_center[1] - cropped_center[1])

    #print("Translated the center in Rotation Matrix to the Cropped Frame Center")

    cropped_aligned_face = cv2.warpAffine(frame,rotation_matrix,(224,224),
                                                      cv2.INTER_CUBIC)

    #print("Cropped and Aligned the face in the Frame")

    cropped_aligned_faces_list.append(cropped_aligned_face)

    print("Preprocessed Frame of {}".format(single_vid_path.parts[-1]))

video.release()

cropped_aligned_faces_list = np.array(cropped_aligned_faces_list)

if "DeepfakeTIMIT" in list(single_vid_path.parts):
    np.savez("./Training Videos/preproc_fake_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)
else:
    np.savez("./Training Videos/preproc_real_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)

print("\n\nProcessed Video at {}".format(single_vid_path))
"""

'\nsingle_vid_path = pathlib.Path("./DeepfakeTIMIT/higher_quality/mrgg0/si569-video-msjs1.avi")\n\nprint("\n\nGoing to preprocess Video at {}".format(single_vid_path))\n\nvideo = cv2.VideoCapture(str(single_vid_path))\ncropped_aligned_faces_list = list()\n\nwhile True:\n\n    is_frame, frame = video.read()\n\n    if not is_frame:\n        break\n\n    print("Going to preprocess Frame of {} having size {}".format(single_vid_path.parts[-1],\n                                                                  frame.shape))\n\n    face_bbox, left_eye_kp, right_eye_kp = detect_face(frame)\n\n    if face_bbox == None:\n        continue\n\n    rotation_angle = determine_rotation_angle(left_eye_kp, right_eye_kp);\n    scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp = determine_scaling_factor((224,224),\n                                                                                            left_eye_kp, right_eye_kp)\n    rotation_center = determine_rotation_center(left_eye_kp, righ

In [ ]:
"""
def preproc_single_vid(single_vid_path):

    print("\n\nGoing to preprocess Video at {}\n".format(single_vid_path))

    video = cv2.VideoCapture(str(single_vid_path))
    cropped_aligned_faces_list = list()

    while True:

        is_frame, frame = video.read()

        if not is_frame:
            break

        print("Going to preprocess frame of {}".format(single_vid_path.parts[-1]))

        face_bbox, left_eye_kp, right_eye_kp = detect_face(frame)
        angle_bw_eyes = determine_rotation_angle(left_eye_kp, right_eye_kp);
        scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp = determine_scaling_factor((224,224),
                                                                                        left_eye_kp, right_eye_kp)
        rotation_center = determine_rotation_center(left_eye_kp, right_eye_kp)

        rotation_matrix = cv2.getRotationMatrix2D(center=rotation_center,
                                                     angle=rotation_angle,scale=scaling_factor)

        cropped_center_x = cropped_left_eye_kp[0]
        cropped_center_y = cropped_left_eye_kp[1] + (np.linalg.norm(x=cropped_right_eye_kp-cropped_right_eye_kp,ord=2)/2)
        cropped_center = (cropped_center_x, cropped_center_y)

        rotation_matrix[0,2] = rotation_matrix[0,2] - (rotation_center[0] - cropped_center[0])
        rotation_matrix[1,2] = rotation_matrix[1,2] - (rotation_center[1] - cropped_center[1])

        cropped_aligned_face = cv2.warpAffine(frame,rotation_matrix,(224,224),
                                                  cv2.INTER_CUBIC)

        cropped_aligned_faces_list.append(cropped_aligned_face)

    video.release()

    cropped_aligned_faces_list = np.array(cropped_aligned_faces_list)

    if "DeepfakeTIMIT" in list(single_vid_path.parts):
        np.savez("./Training Videos/preproc_fake_vids/{}.npz".format(single_vid_path.parts[-1]),
                         args=cropped_aligned_faces_list)
    else:
        np.savez("./Training Videos/preproc_real_vids/{}.npz".format(single_vid_path.parts[-1]),
                         args=cropped_aligned_faces_list)

    print("\n\nProcessed Video at {}".format(single_vid_path))
"""

'\ndef preproc_single_vid(single_vid_path):\n\n    print("\n\nGoing to preprocess Video at {}\n".format(single_vid_path))\n\n    video = cv2.VideoCapture(str(single_vid_path))\n    cropped_aligned_faces_list = list()\n\n    while True:\n\n        is_frame, frame = video.read()\n\n        if not is_frame:\n            break\n\n        print("Going to preprocess frame of {}".format(single_vid_path.parts[-1]))\n\n        face_bbox, left_eye_kp, right_eye_kp = detect_face(frame)\n        angle_bw_eyes = determine_rotation_angle(left_eye_kp, right_eye_kp);\n        scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp = determine_scaling_factor((224,224),\n                                                                                        left_eye_kp, right_eye_kp)\n        rotation_center = determine_rotation_center(left_eye_kp, right_eye_kp)\n\n        rotation_matrix = cv2.getRotationMatrix2D(center=rotation_center,\n                                                     angle=rota

In [ ]:
def preproc_single_vid(mb_vids_paths):

    for single_vid_path in mb_vids_paths:

        print("\n\nGoing to preprocess Video at {}".format(single_vid_path))

        video = cv2.VideoCapture(str(single_vid_path))
        cropped_aligned_faces_list = list()

        while True:

            is_frame, frame = video.read()

            if not is_frame:
                break
            """
            print("Going to preprocess Frame of {} having size {}".format(single_vid_path.parts[-1],
                                                                  frame.shape))
            """
            face_bbox, left_eye_kp, right_eye_kp = detect_face(frame)

            if face_bbox == None:
                continue

            rotation_angle = determine_rotation_angle(left_eye_kp, right_eye_kp);
            scaling_factor, cropped_left_eye_kp, cropped_right_eye_kp = determine_scaling_factor((224,224),
                                                                                            left_eye_kp, right_eye_kp)
            rotation_center = determine_rotation_center(left_eye_kp, right_eye_kp)

            #print("Computed everything to get the Rotation Matrix for the Frame")

            rotation_matrix = cv2.getRotationMatrix2D(center=rotation_center,
                                                         angle=rotation_angle,scale=scaling_factor)

            #print("Computed the Rotation Matrix for the Frame")

            cropped_center_x = cropped_left_eye_kp[0]
            cropped_center_y = cropped_left_eye_kp[1] + (np.linalg.norm(x=cropped_right_eye_kp-cropped_right_eye_kp,ord=2)/2)
            cropped_center = (cropped_center_x, cropped_center_y)

            rotation_matrix[0,2] = rotation_matrix[0,2] - (rotation_center[0] - cropped_center[0])
            rotation_matrix[1,2] = rotation_matrix[1,2] - (rotation_center[1] - cropped_center[1])

            #print("Translated the center in Rotation Matrix to the Cropped Frame Center")

            cropped_aligned_face = cv2.warpAffine(frame,rotation_matrix,(224,224),
                                                      cv2.INTER_CUBIC)

            #print("Cropped and Aligned the face in the Frame")

            cropped_aligned_faces_list.append(cropped_aligned_face)

            print("Preprocessed Frame of {}".format(single_vid_path.parts[-1]))

        video.release()

        cropped_aligned_faces_list = np.array(cropped_aligned_faces_list)

        if "DeepfakeTIMIT" in list(single_vid_path.parts):
            np.savez("./Training Videos/preproc_fake_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)
        else:
            np.savez("./Training Videos/preproc_real_vids/{}.npz".format(str(single_vid_path.parts[-1]).replace(".avi","")),
                             args=cropped_aligned_faces_list)

        print("\n\nProcessed Video at {}".format(single_vid_path))

In [ ]:
from concurrent.futures import ThreadPoolExecutor
from collections import deque

vid_counter = 0
mb_number = 0

real_fake_preproc_vids_paths = ["./Training Videos/preproc_fake_vids",
                               "./Training Videos/preproc_real_vids"]

for vids_path, preproc_vids_path in zip(real_fake_vids_paths,
                                                    real_fake_preproc_vids_paths):

    vids2preproc_path_list = deque()

    for single_vid_path in pathlib.Path(vids_path).glob("*/*.avi"):

        vids2preproc_path_list.append(single_vid_path)
        vid_counter += 1

        if vid_counter % (os.cpu_count()) == 0:

            start_time = time.time()

            with ThreadPoolExecutor(max_workers=(os.cpu_count())) as pool:
                future = pool.submit(preproc_single_vid, list(vids2preproc_path_list))
                future.result()

            end_time = time.time()
            mb_number += 1
            elapsed_time = end_time - start_time

            print("\n\nProcessed Mini Batch # {} of 64 Videos in {} seconds".format(mb_number,elapsed_time))

            vids2preproc_path_list.clear()




Going to preprocess Video at DeepfakeTIMIT/higher_quality/fkms0/si860-video-fjem0.avi
1/1 [==============================] - 0s 245ms/step
Preprocessed Frame of si860-video-fjem0.avi
1/1 [==============================] - 0s 89ms/step


1/1 [==============================] - 0s 255ms/step
Preprocessed Frame of si860-video-fjem0.avi
1/1 [==============================] - 0s 84ms/step


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 126ms/step
Preprocessed Frame of sa1-video-fadg0.avi
1/1 [==============================] - 0s 162ms/step
Preprocessed Frame of sa1-video-fadg0.avi
1/1 [==============================] - 0s 176ms/step
Preprocessed Frame of sa1-video-fadg0.avi
1/1 [==============================] - 0s 131ms/step
Preprocessed Frame of sa1-video-fadg0.avi
1/1 [==============================] - 0s 125ms/step
Preprocessed Frame of sa1-video-fadg0.avi
1/1 [==============================] - 0s 139ms/step
Preprocessed Frame of sa1-video-fadg0.avi
1/1 [==============================] - 0s 121ms/step
Preprocessed Frame of sa1-video-fadg0.avi
1/1 [==============================] - 0s 129ms/step
Preprocessed Frame of sa1-video-fadg0.avi
1/1 [==============================] - 0s 128ms/step
Preprocessed Frame of sa1-video-fadg0.avi
1/1 [==============================] - 0s 126ms/step
Preprocessed Frame of sa1-video-fadg0.avi

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-30-674ee3a188d0>", line 26, in <cell line: 10>
    future.result()
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 453, in result
    self._condition.wait(timeout)
  File "/usr/lib/python3.10/threading.py", line 320, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-30-674ee3a188d0>", line 24, in <cell line: 10>
    with ThreadPoolExecutor(max_workers=(os.cpu_count())) as pool:
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 649, in __exit__
    self.shutdown(wait=True)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 235, in shutdown
    t.join()
  File "/usr/lib/python3.10/threading.py", line 1096, in j

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "<ipython-input-30-674ee3a188d0>", line 26, in <cell line: 10>
    future.result()
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 453, in result
    self._condition.wait(timeout)
  File "/usr/lib/python3.10/threading.py", line 320, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-30-674ee3a188d0>", line 24, in <cell line: 10>
    with ThreadPoolExecutor(max_workers=(os.cpu_count())) as pool:
  File "/usr/lib/python3.10/concurrent/futures/_base.py", line 649, in __exit__
    self.shutdown(wait=True)
  File "/usr/lib/python3.10/concurrent/futures/thread.py", line 235, in shutdown
    t.join()
  File "/usr/lib/python3.10/threading.py", line 1096, in j

TypeError: object of type 'NoneType' has no len()

Preprocessed Frame of sx370-video-fadg0.avi


Taking already preprocessed npz files above from drive which I have already saved.

In [4]:
!cp /content/drive/MyDrive/Project\ Data/Training\ Videos.zip /content


In [5]:
!rm -rf "/content/Training Videos/preproc_fake_vids"
!rm -rf "/content/Training Videos/preproc_real_vids"
!rm -rf "/content/Training Videos"

In [6]:
!unzip -qq "/content/Training Videos.zip"

In [144]:
training_real_vids_paths = list()
training_fake_vids_paths = list()

for single_img_path in pathlib.Path("./Training Videos").glob("*/*.npz"):

    if "real" in str(single_img_path.parts[-2]).split("_"):
        training_real_vids_paths.append(str(single_img_path))

    elif "fake" in str(single_img_path.parts[-2]).split("_"):
        training_fake_vids_paths.append(str(single_img_path))

In [145]:
def custom_training_data_generator(mb_size):

    for i in range(len(training_fake_vids_paths)//2):

        X_train_mb = []
        Y_train_mb = []

        training_fake_mb_vids_paths = training_fake_vids_paths[i*(mb_size//2):(i+1)*(mb_size//2)]
        training_real_mb_vids_paths = training_real_vids_paths[i*(mb_size//2):(i+1)*(mb_size//2)]

        for single_fake_vid_path,single_real_vid_path in zip(training_fake_mb_vids_paths,
                                                             training_real_mb_vids_paths):
            X_train_mb.append(np.load(file=single_real_vid_path)['args'])
            X_train_mb.append(np.load(file=single_fake_vid_path)['args'])

            Y_train_mb.append(0)
            Y_train_mb.append(1)

        X_train_mb = np.concatenate(X_train_mb,axis=0)
        Y_train_mb = np.array(Y_train_mb)

        yield X_train_mb,Y_train_mb

In [146]:
train_datagen = custom_training_data_generator(4)

In [147]:
X_train_mb, Y_train_mb = next(train_datagen)

In [148]:
X_train_mb.shape

(476, 224, 224, 3)

In [149]:
Y_train_mb.shape

(4,)

In [ ]:

# # Reshape X_train_mb to have an additional dimension for the sequence length
# X_train_mb = X_train_mb.reshape(-1, 476, 224, 224, 3)
# X_train_mb.shape

In [48]:
# # Reshape Y_train_mb to have the same number of samples as X_train_mb
# Y_train_mb = np.tile(Y_train_mb, (len(X_train_mb), 1))

# # Now both X_train_mb and Y_train_mb have the same number of samples

In [66]:
# # Determine the minimum number of samples between X_train_mb and Y_train_mb
# min_samples = min(len(X_train_mb), len(Y_train_mb))

# # Truncate X_train_mb and Y_train_mb to have the same number of samples
# X_train_mb = X_train_mb[:min_samples]
# Y_train_mb = Y_train_mb[:min_samples]

In [161]:
X_train_mb_reshaped = X_train_mb.reshape(-1, 4, 224, 224, 3)
Y_train_mb_expanded = np.repeat(Y_train_mb, repeats=119)

In [163]:
X_train_mb_reshaped.shape

(119, 4, 224, 224, 3)

In [164]:
Y_train_mb_expanded.shape

(476,)

In [165]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import TimeDistributed, LSTM, Dense, Flatten, Input
from tensorflow.keras.applications import ResNet50

In [166]:
def create_model():

    base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False

    inputs = Input(shape=(4, 224, 224, 3))

    x = TimeDistributed(base_model)(inputs)


    x = TimeDistributed(Flatten())(x)

    x = LSTM(50, return_sequences=False)(x)

    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs=inputs, outputs=outputs)

    return model

In [167]:
model = create_model()

In [168]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [169]:
model.summary()

Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_22 (InputLayer)       [(None, 4, 224, 224, 3)   0         
                             ]                                   
                                                                 
 time_distributed_18 (TimeD  (None, 4, 7, 7, 2048)     23587712  
 istributed)                                                     
                                                                 
 time_distributed_19 (TimeD  (None, 4, 100352)         0         
 istributed)                                                     
                                                                 
 lstm_8 (LSTM)               (None, 50)                20080600  
                                                                 
 dense_8 (Dense)             (None, 1)                 51        
                                                           

In [175]:
number_of_batches = len(X_train_mb_reshaped) // 32
model.fit(X_train_mb_reshaped, Y_train_mb_expanded, epochs=10, steps_per_epoch= number_of_batches, batch_size=32, validation_split=0.2)

Epoch 1/10
3/3 [==============================] - 89s 36s/step - loss: 0.0109 - accuracy: 1.0000 - val_loss: 0.0120 - val_accuracy: 1.0000
Epoch 2/10
3/3 [==============================] - 69s 26s/step - loss: 0.0104 - accuracy: 1.0000 - val_loss: 0.0115 - val_accuracy: 1.0000
Epoch 3/10
3/3 [==============================] - 69s 26s/step - loss: 0.0097 - accuracy: 1.0000 - val_loss: 0.0110 - val_accuracy: 1.0000
Epoch 4/10
3/3 [==============================] - 69s 26s/step - loss: 0.0093 - accuracy: 1.0000 - val_loss: 0.0106 - val_accuracy: 1.0000
Epoch 5/10
3/3 [==============================] - 69s 26s/step - loss: 0.0088 - accuracy: 1.0000 - val_loss: 0.0102 - val_accuracy: 1.0000
Epoch 6/10
3/3 [==============================] - 69s 26s/step - loss: 0.0084 - accuracy: 1.0000 - val_loss: 0.0099 - val_accuracy: 1.0000
Epoch 7/10
3/3 [==============================] - 69s 26s/step - loss: 0.0081 - accuracy: 1.0000 - val_loss: 0.0095 - val_accuracy: 1.0000
Epoch 8/10
3/3 [===========